In [97]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [98]:
df = pd.read_csv('../../data/obesity_transform.csv')
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,Bad_Habits,CALC,MTRANS,NObeyesdad
0,Female,21,162.0,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,1,no,Public_Transportation,Normal_Weight
1,Male,23,180.0,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,1,Frequently,Public_Transportation,Normal_Weight
2,Male,27,180.0,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,1,Frequently,Walking,Overweight_Level_I
3,Male,22,178.0,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,1,Sometimes,Public_Transportation,Overweight_Level_II
4,Male,29,162.0,53.0,no,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,0.0,1,Sometimes,Automobile,Normal_Weight


In [99]:
df.isnull().sum()

Gender                            3
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
Bad_Habits                        0
CALC                              0
MTRANS                            5
NObeyesdad                        0
dtype: int64

In [100]:
vcat = ["family_history_with_overweight", "FAVC", "CAEC", "SMOKE", "SCC", "CALC", "NObeyesdad"]
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA

# Asumiendo que df es tu DataFrame y 'Gender' y 'MTRANS' son las columnas con valores nulos

# Preparar los datos
data1 = df[df['Gender'].notna()]
data2 = df[df['MTRANS'].notna()]

# Rellenar los valores NaN
imputer = SimpleImputer(strategy='most_frequent')
data1[vcat] = imputer.fit_transform(data1[vcat])
data2[vcat] = imputer.fit_transform(data2[vcat])

# Codificar las variables categóricas
encoder = OrdinalEncoder()
data1[vcat] = encoder.fit_transform(data1[vcat])
data2[vcat] = encoder.fit_transform(data2[vcat])

# Dividir los datos en entrenamiento y prueba
train1, test1 = train_test_split(data1, test_size=0.2, random_state=42)
train2, test2 = train_test_split(data2, test_size=0.2, random_state=42)

# Crear el modelo para la primera columna
model1 = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, max_features='sqrt')
model1.fit(train1.drop(['Gender', 'MTRANS'], axis=1), train1['Gender'])

# Crear el modelo para la segunda columna
model2 = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, max_features='sqrt')
model2.fit(train2.drop(['Gender', 'MTRANS'], axis=1), train2['MTRANS'])

# Calcular las predicciones en los datos de prueba
predicted1_test = model1.predict(test1.drop(['Gender', 'MTRANS'], axis=1))
predicted2_test = model2.predict(test2.drop(['Gender', 'MTRANS'], axis=1))

# Calcular y mostrar las métricas del modelo
print("Model 1 Accuracy:", metrics.accuracy_score(test1['Gender'], predicted1_test))
print("Model 2 Accuracy:", metrics.accuracy_score(test2['MTRANS'], predicted2_test))

# Calcular y mostrar las métricas de entrenamiento
predicted1_train = model1.predict(train1.drop(['Gender', 'MTRANS'], axis=1))
predicted2_train = model2.predict(train2.drop(['Gender', 'MTRANS'], axis=1))

print("Model 1 Train Accuracy:", metrics.accuracy_score(train1['Gender'], predicted1_train))
print("Model 2 Train Accuracy:", metrics.accuracy_score(train2['MTRANS'], predicted2_train))

# Calcular y mostrar las métricas de validación cruzada
cv_score1 = cross_val_score(model1, train1.drop(['Gender', 'MTRANS'], axis=1), train1['Gender'], cv=5)
cv_score2 = cross_val_score(model2, train2.drop(['Gender', 'MTRANS'], axis=1), train2['MTRANS'], cv=5)

print("Model 1 CV Score:", cv_score1.mean())
print("Model 2 CV Score:", cv_score2.mean())

# Imputar los valores nulos en el DataFrame original
df_nulls1 = df[df['Gender'].isna()]
df_nulls2 = df[df['MTRANS'].isna()]

df_nulls1[vcat] = imputer.transform(df_nulls1[vcat])
df_nulls1[vcat] = encoder.transform(df_nulls1[vcat])

df_nulls2[vcat] = imputer.transform(df_nulls2[vcat])
df_nulls2[vcat] = encoder.transform(df_nulls2[vcat])

predicted1 = model1.predict(df_nulls1.drop(['Gender', 'MTRANS'], axis=1))
predicted2 = model2.predict(df_nulls2.drop(['Gender', 'MTRANS'], axis=1))

df.loc[df['Gender'].isna(), 'Gender'] = predicted1
df.loc[df['MTRANS'].isna(), 'MTRANS'] = predicted2

Model 1 Accuracy: 0.9437939110070258
Model 2 Accuracy: 0.9110070257611241
Model 1 Train Accuracy: 0.9923798358733881
Model 2 Train Accuracy: 0.9618544600938967
Model 1 CV Score: 0.9337792183293032
Model 2 CV Score: 0.906103156805244


Se han utilizado modelos de clasificación para rellenar los nulos de las columnas "Gender" y "MTRANS". Las conclusiones de las métricas de los modelos son las siguientes:

1. Model 1 Accuracy: Esta es la precisión de tu primer modelo en los datos de prueba. Significa que el modelo predijo correctamente el género en aproximadamente el 94.38% de los casos en el conjunto de prueba.

2. Model 2 Accuracy: Esta es la precisión de tu segundo modelo en los datos de prueba. Significa que el modelo predijo correctamente el método de transporte en aproximadamente el 91.10% de los casos en el conjunto de prueba.

3. Model 1 Train Accuracy: Esta es la precisión de tu primer modelo en los datos de entrenamiento. Un valor de 0.9935 significa que el modelo predijo correctamente el género en aproximadamente el 99.24% de los casos en el conjunto de entrenamiento.

4. Model 2 Train Accuracy: Esta es la precisión de tu segundo modelo en los datos de entrenamiento. Un valor de 0.9695 significa que el modelo predijo correctamente el método de transporte en aproximadamente el 96.19% de los casos en el conjunto de entrenamiento.

5. Model 1 CV Score: Este es el puntaje de validación cruzada de tu primer modelo. La validación cruzada es una técnica que divide tus datos en un número de 'pliegues', y luego entrena y prueba tu modelo en diferentes subconjuntos de esos pliegues. Un puntaje de validación cruzada de 0.9297 significa que, en promedio, tu modelo tuvo una precisión del 93.38% en estos conjuntos de prueba de validación cruzada.

6. Model 2 CV Score: Este es el puntaje de validación cruzada de tu segundo modelo. Un puntaje de validación cruzada de 0.9055 significa que, en promedio, tu modelo tuvo una precisión del 90.61% en estos conjuntos de prueba de validación cruzada.


In [101]:
df.isnull().sum()

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
Bad_Habits                        0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [104]:
df.columns = df.columns.str.lower()

In [105]:
df.head()

,gender,age,height,weight,family_history_with_overweight,favc,fcvc,ncp,caec,smoke,ch2o,scc,faf,tue,bad_habits,calc,mtrans,nobeyesdad
0,Female,21,162.0,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,1,no,Public_Transportation,Normal_Weight
1,Male,23,180.0,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,1,Frequently,Public_Transportation,Normal_Weight
2,Male,27,180.0,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,1,Frequently,Walking,Overweight_Level_I
3,Male,22,178.0,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,1,Sometimes,Public_Transportation,Overweight_Level_II
4,Male,29,162.0,53.0,no,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,0.0,1,Sometimes,Automobile,Normal_Weight


In [107]:
df.to_csv('../../data/obesity_final.csv', index=False)